In [47]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt 
import os 
import cv2

In [32]:
# Step 1: Load the ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False)

In [33]:
# Step 2: Add custom layers for binary classification (cat vs dog)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Use 'sigmoid' for binary classification
model = Model(inputs=base_model.input, outputs=predictions)

# Step 3: Set up the model for fine-tuning
# Initially, freeze all layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./data/',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory('./data/',
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        class_mode='binary')

# Step 5: Train the model
model.fit(train_generator,
          epochs = 10,
          validation_data=validation_generator)



Found 140 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Epoch 1/10


2023-10-14 14:59:33.929953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - ETA: 0s - loss: 1.7768 - accuracy: 0.4143

2023-10-14 14:59:49.864793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 29s 5s/step - loss: 1.7768 - accuracy: 0.4143 - val_loss: 2.0328 - val_accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 3s 626ms/step - loss: 1.6130 - accuracy: 0.5000 - val_loss: 0.7292 - val_accuracy: 0.5000
Epoch 3/10
5/5 [==============================] - 3s 529ms/step - loss: 1.2854 - accuracy: 0.5000 - val_loss: 0.8684 - val_accuracy: 0.5000
Epoch 4/10
5/5 [==============================] - 3s 526ms/step - loss: 0.9346 - accuracy: 0.5071 - val_loss: 1.1492 - val_accuracy: 0.5000
Epoch 5/10
5/5 [==============================] - 3s 570ms/step - loss: 0.8387 - accuracy: 0.4571 - val_loss: 0.8678 - val_accuracy: 0.5000
Epoch 6/10
5/5 [==============================] - 3s 541ms/step - loss: 0.8173 - accuracy: 0.4929 - val_loss: 0.6876 - val_accuracy: 0.5286
Epoch 7/10
5/5 [==============================] - 3s 595ms/step - loss: 0.6754 - accuracy: 0.5571 - val_loss: 0.6487 - val_accuracy: 0.5643
Epoch 8/10
5/5 [=================

In [34]:
from tensorflow.keras.preprocessing import image

def load_and_preprocess_image(img_path):
    """Load and preprocess the image for prediction."""
    # Load the image with the target size that your model expects
    img = image.load_img(img_path, target_size=(224, 224))
    # Convert the PIL image to a numpy array
    img_array = image.img_to_array(img)
    # Expand the shape of the array from (224, 224, 3) to (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    # Preprocess the image to make it match the format ResNet was originally trained on
    img_array /= 255.0
    return img_array

# Load and preprocess the image
img_path = './data/cats/cat_113.jpg'
img_array = load_and_preprocess_image(img_path)

# Make the prediction
predictions = model.predict(img_array)

# Interpret the prediction
if predictions[0][0] > 0.5:
    print("It's a dog!")
else:
    print("It's a cat!")

2023-10-14 15:00:25.846498: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step
It's a cat!
